In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
data = [
    [101, 'John Bell', 'UT', 'C', 'jb', 6,'7522MB'],
    [101, 'John Derek', 'UT', 'B', 'jb', 7,'7523MB'],
    [101, 'John Bell', 'UT', 'C', 'jc', 8,'7522MG'],
    [101, 'John Derek', 'AMS', 'C', 'jd', 8,'7522MD'],
    [102, 'Claire Stevens', 'UT', 'F', 'cs', 10,'7532MB'],
    [102, 'Claire Grey', 'UT', 'F', 'cg', 10,'7522MB'],
    [103, 'Pet Nortan', 'EIND', 'G', 'pn', 11,'7525NB'],
    [103, 'Pet Nortan', 'EIND', 'G', 'pn', 11,'7525NB'],
    [104, 'George Bailey', 'EIND', 'G', 'sn',12, '7525NB'],
    [104, 'George Baily', 'EIND', 'G', 'sn',12, '7525NB'],
    [104, 'Jeorge Baily', 'EIND', 'G', 'sn',12, '7525NB'],
    [105, 'Leah Murphy', 'AMS', 'G', 'gn', 13,'7425NB']
]

columns = ['SID','SName','University','Address','EmailId','PhoneNum','Postcode']

df_data = pd.DataFrame(data, columns=columns)
print(df_data)

    SID           SName University Address EmailId  PhoneNum Postcode
0   101       John Bell         UT       C      jb         6   7522MB
1   101      John Derek         UT       B      jb         7   7523MB
2   101       John Bell         UT       C      jc         8   7522MG
3   101      John Derek        AMS       C      jd         8   7522MD
4   102  Claire Stevens         UT       F      cs        10   7532MB
5   102     Claire Grey         UT       F      cg        10   7522MB
6   103      Pet Nortan       EIND       G      pn        11   7525NB
7   103      Pet Nortan       EIND       G      pn        11   7525NB
8   104   George Bailey       EIND       G      sn        12   7525NB
9   104    George Baily       EIND       G      sn        12   7525NB
10  104    Jeorge Baily       EIND       G      sn        12   7525NB
11  105     Leah Murphy        AMS       G      gn        13   7425NB


In [4]:
from src.database.db_connection import *

In [5]:
engine = create_engine_for_db()

In [6]:
df_data.to_sql('students',engine,schema='prototype1',index=False,if_exists='replace')

12

In [7]:
fd_constraint1 = [(['SID'],['SName'])]
#fd_constraint1 = [(['SID','University'],['SName'])]

# Step 1 - simplify FDs to have one attribute at RHS

In [8]:
from src.utilities.step1_simplify_fds import *

In [9]:
fd_constraint1 = simplify_FDs(fd_constraint1)

In [10]:
fd_constraint1

[(['SID'], ['SName'])]

# Step 2: clusters + generating possible repairs per cluster

In [11]:
from src.utilities.step2_clustering_and_generating_repairs import *

### Step 2_0: Preprocess data by adding a new column with unique identifier

In [12]:
df_data = preprocess_data(df_data)

In [13]:
df_data

,SID,SName,University,Address,EmailId,PhoneNum,Postcode,uuid
0,101,John Bell,UT,C,jb,6,7522MB,1
1,101,John Derek,UT,B,jb,7,7523MB,2
2,101,John Bell,UT,C,jc,8,7522MG,3
3,101,John Derek,AMS,C,jd,8,7522MD,4
4,102,Claire Stevens,UT,F,cs,10,7532MB,5
5,102,Claire Grey,UT,F,cg,10,7522MB,6
6,103,Pet Nortan,EIND,G,pn,11,7525NB,7
7,104,George Bailey,EIND,G,sn,12,7525NB,8
8,104,George Baily,EIND,G,sn,12,7525NB,9
9,104,Jeorge Baily,EIND,G,sn,12,7525NB,10


### Step 2_1: Identify clusters - group by FD_LHS

In [14]:
# src.utilities.step2_clustering_and_generating_repair identify_clusters

### Step 2_2: Find minimum actions needed within a cluster

In [15]:
# src.utilities.step2_clustering_and_generating_repair find_minimum_actions

### Step 2_3: Define highest frequency sub-clusters

In [16]:
# src.utilities.step2_clustering_and_generating_repair mark_high_frequence_subclusters

### Step 2_4: Create and assign RVs for delete, update & delete/update cases

In [17]:
# src.utilities.step2_clustering_and_generating_repair create_rv_definitions

In [18]:
rv_definitions,rv_probabilities,consistent_rows = create_rv_definitions(df_data,fd_constraint1)

In [19]:
rv_definitions

{'fd1SName1=1': ('John Bell', {1, 3}),
 'fd1SName1=2': ('John Bell', {1, 2, 3}),
 'fd1SName1=3': ('John Bell', {1, 3, 4}),
 'fd1SName1=4': ('John Bell', {1, 2, 3, 4}),
 'fd1SName1=5': ('John Derek', {2, 4}),
 'fd1SName1=6': ('John Derek', {1, 2, 4}),
 'fd1SName1=7': ('John Derek', {2, 3, 4}),
 'fd1SName1=8': ('John Derek', {1, 2, 3, 4}),
 'fd1SName2=1': ('Claire Grey', {6}),
 'fd1SName2=2': ('Claire Grey', {5, 6}),
 'fd1SName2=3': ('Claire Stevens', {5}),
 'fd1SName2=4': ('Claire Stevens', {5, 6}),
 'fd1SName4=1': ('George Bailey', {8}),
 'fd1SName4=2': ('George Bailey', {8, 9}),
 'fd1SName4=3': ('George Bailey', {8, 10}),
 'fd1SName4=4': ('George Bailey', {8, 9, 10}),
 'fd1SName4=5': ('George Baily', {9}),
 'fd1SName4=6': ('George Baily', {8, 9}),
 'fd1SName4=7': ('George Baily', {9, 10}),
 'fd1SName4=8': ('George Baily', {8, 9, 10}),
 'fd1SName4=9': ('Jeorge Baily', {10}),
 'fd1SName4=10': ('Jeorge Baily', {8, 10}),
 'fd1SName4=11': ('Jeorge Baily', {9, 10}),
 'fd1SName4=12': ('Jeorg

In [20]:
rv_probabilities

{'fd1SName1=1': 0.125,
 'fd1SName1=2': 0.125,
 'fd1SName1=3': 0.125,
 'fd1SName1=4': 0.125,
 'fd1SName1=5': 0.125,
 'fd1SName1=6': 0.125,
 'fd1SName1=7': 0.125,
 'fd1SName1=8': 0.125,
 'fd1SName2=1': 0.25,
 'fd1SName2=2': 0.25,
 'fd1SName2=3': 0.25,
 'fd1SName2=4': 0.25,
 'fd1SName4=1': 0.08333333333333333,
 'fd1SName4=2': 0.08333333333333333,
 'fd1SName4=3': 0.08333333333333333,
 'fd1SName4=4': 0.08333333333333333,
 'fd1SName4=5': 0.08333333333333333,
 'fd1SName4=6': 0.08333333333333333,
 'fd1SName4=7': 0.08333333333333333,
 'fd1SName4=8': 0.08333333333333333,
 'fd1SName4=9': 0.08333333333333333,
 'fd1SName4=10': 0.08333333333333333,
 'fd1SName4=11': 0.08333333333333333,
 'fd1SName4=12': 0.08333333333333333}

In [21]:
consistent_rows

[7, 11]

# Step 3: Convert back to repaired database table

In [22]:
from src.utilities.step3_convert_back_to_db import *

In [23]:
repaired_rows = create_repaired_rows(rv_definitions,df_data,consistent_rows)

In [24]:
columns = [col for col in df_data.columns if col != 'uuid'] + ['_sentences']
repaired_df = pd.DataFrame(repaired_rows, columns=columns)

In [25]:
repaired_df

,SID,SName,University,Address,EmailId,PhoneNum,Postcode,_sentences
0,101,John Bell,UT,C,jb,6,7522MB,fd1SName1=1 | fd1SName1=2 | fd1SName1=3 | fd1S...
1,101,John Bell,UT,C,jc,8,7522MG,fd1SName1=1 | fd1SName1=2 | fd1SName1=3 | fd1S...
2,101,John Bell,UT,B,jb,7,7523MB,fd1SName1=2 | fd1SName1=4
3,101,John Bell,AMS,C,jd,8,7522MD,fd1SName1=3 | fd1SName1=4
4,101,John Derek,UT,B,jb,7,7523MB,fd1SName1=5 | fd1SName1=6 | fd1SName1=7 | fd1S...
5,101,John Derek,AMS,C,jd,8,7522MD,fd1SName1=5 | fd1SName1=6 | fd1SName1=7 | fd1S...
6,101,John Derek,UT,C,jb,6,7522MB,fd1SName1=6 | fd1SName1=8
7,101,John Derek,UT,C,jc,8,7522MG,fd1SName1=7 | fd1SName1=8
8,102,Claire Grey,UT,F,cg,10,7522MB,fd1SName2=1 | fd1SName2=2
9,102,Claire Grey,UT,F,cs,10,7532MB,fd1SName2=2


# Step 4: Merge same rows by combining sentences

In [26]:
from src.utilities.step4_group_similar_sentences import *

In [27]:
repaired_df = group_similar_rows_together(repaired_df)

In [28]:
repaired_df

,SID,SName,University,Address,EmailId,PhoneNum,Postcode,_sentences
0,101,John Bell,AMS,C,jd,8,7522MD,fd1SName1=3 | fd1SName1=4
1,101,John Bell,UT,B,jb,7,7523MB,fd1SName1=2 | fd1SName1=4
2,101,John Bell,UT,C,jb,6,7522MB,fd1SName1=1 | fd1SName1=2 | fd1SName1=3 | fd1S...
3,101,John Bell,UT,C,jc,8,7522MG,fd1SName1=1 | fd1SName1=2 | fd1SName1=3 | fd1S...
4,101,John Derek,AMS,C,jd,8,7522MD,fd1SName1=5 | fd1SName1=6 | fd1SName1=7 | fd1S...
5,101,John Derek,UT,B,jb,7,7523MB,fd1SName1=5 | fd1SName1=6 | fd1SName1=7 | fd1S...
6,101,John Derek,UT,C,jb,6,7522MB,fd1SName1=6 | fd1SName1=8
7,101,John Derek,UT,C,jc,8,7522MG,fd1SName1=7 | fd1SName1=8
8,102,Claire Grey,UT,F,cg,10,7522MB,fd1SName2=1 | fd1SName2=2
9,102,Claire Grey,UT,F,cs,10,7532MB,fd1SName2=2


# Step 5: Insert in Dubio

In [29]:
repaired_df.to_sql('students_prob',engine,schema='prototype1',index=False,if_exists='replace')

17

In [32]:
entries = [f"{k}:{v}" for k, v in rv_probabilities.items()]
joined = ';'.join(entries)

update_stmt = f"UPDATE prototype1._dict\nSET dict=add(dict, '{joined}')\nWHERE name='students_dict';"

print(update_stmt)

UPDATE prototype1._dict
SET dict=add(dict, 'fd1SName1=1:0.125;fd1SName1=2:0.125;fd1SName1=3:0.125;fd1SName1=4:0.125;fd1SName1=5:0.125;fd1SName1=6:0.125;fd1SName1=7:0.125;fd1SName1=8:0.125;fd1SName2=1:0.25;fd1SName2=2:0.25;fd1SName2=3:0.25;fd1SName2=4:0.25;fd1SName4=1:0.08333333333333333;fd1SName4=2:0.08333333333333333;fd1SName4=3:0.08333333333333333;fd1SName4=4:0.08333333333333333;fd1SName4=5:0.08333333333333333;fd1SName4=6:0.08333333333333333;fd1SName4=7:0.08333333333333333;fd1SName4=8:0.08333333333333333;fd1SName4=9:0.08333333333333333;fd1SName4=10:0.08333333333333333;fd1SName4=11:0.08333333333333333;fd1SName4=12:0.08333333333333333')
WHERE name='students_dict';


In [33]:
with engine.begin() as connection:
    connection.execute(text(update_stmt))